## Agents and tools using LCEL

In [ ]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0, 
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)


### create_react_agent

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor, tool
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain import hub


@tool
def addition(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

tools = [addition]

# Custom prompt with explicit JSON formatting instructions
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action as a valid JSON object. For example: {{"a": 10, "b": 5}}
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

CRITICAL: Action Input MUST be valid JSON with the exact parameter names from the tool schema.

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(google_llm, tools, prompt=prompt)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True  # Important!
)

res = agent_executor.invoke({"input": "Add 10 and 5"})
print(res)

### create_tool_calling_agent

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor, tool
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain import hub


# class AdditionInput(BaseModel):
#     a: int = Field(description="First number")
#     b: int = Field(description="Second number")

# @tool("addition", args_schema=AdditionInput)

@tool
def addition(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@tool
def uppercase(input: str) -> str:
    """Convert string to uppercase"""
    return input.upper()

@tool
def general_query(input: str) -> str:
    """General question for llm"""
    return google_llm.invoke(input)

tools = [addition, uppercase, general_query]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Use the newer tool-calling agent
agent = create_tool_calling_agent(google_llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

res = agent_executor.invoke({"input": "Add 10 and 5 and convert 'praveen' to uppercase and what is the age of MS Dhoni in 2025?"})

res

### create_structured_chat_agent
##### Works on google_llm but iterates over and over for openai_llm

In [ ]:
from langchain.agents import create_structured_chat_agent, AgentExecutor, tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# Define input schema for structured agent
class AdditionInput(BaseModel):
    a: int = Field(description="First number to add")
    b: int = Field(description="Second number to add")

@tool("addition", args_schema=AdditionInput)
def addition(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

tools = [addition]

prompt = ChatPromptTemplate.from_messages([
    ("system", """Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final answer to the original input question"
}}
```

Begin!

{agent_scratchpad}"""),
    ("human", "{input}"),
])

agent = create_structured_chat_agent(google_llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

res = agent_executor.invoke({"input": "Add 10 and 5"})
print(res)